In [ ]:
import sys
if 'google.colab' not in sys.modules:
    raise OSError("This notebook works only on colab (for now)")

In [ ]:
!sudo apt update > /dev/null
!sudo apt install libcairo2-dev ffmpeg texlive texlive-latex-extra texlive-fonts-extra texlive-latex-recommended texlive-science tipa libpango1.0-dev > /dev/null
!pip install manim > /dev/null
!pip install IPython --upgrade > /dev/null

In [ ]:
from pathlib import Path
import random

import IPython 
from IPython.display import display, Video, HTML

from manim import *;

In [ ]:
# TODO: seting the config like this might be awful
config.verbosity = "ERROR"
config.quality = "example_quality"

def render(scene: Scene):
    "Macro that renders and displays the scene"
    scene().render(preview=True)
    video = Video(
        Path(config["output_file"]).relative_to(Path.cwd()),
        html_attributes='controls autoplay style="max-width: 100%;max-height:500px"',
        embed=True,
    )
    display(HTML(f"""<div style="text-align:center">{video._repr_html_()}</div>"""))
    return scene

class BaseScene(Scene):
    def prepare_tex(self):
        self.__tex_template = TexTemplate(documentclass=R"\documentclass[preview,spanish]{standalone}")
        self.__tex_template.add_to_preamble(R"\usepackage[utf8]{inputenc}")
        self.__tex_template.add_to_preamble(R"\usepackage{enumitem}")
        self.__tex_template.add_to_preamble(R"\usepackage{babel}")
        self.__tex_template.add_to_preamble(R"\setenumerate{noitemsep}")

    @property
    def tex_kwargs(self):
        return {'tex_template': self.__tex_template}

    def construct(self):
        self.prepare_tex()
        self.make()
    
    def make(self):
        raise NotImplementedError()

In [ ]:
%%manim TitleScene
class TitleScene(BaseScene):
    def make(self):
        random.seed(0)
        title = Tex(R"Heurística de\\Conflictos Lineales", **self.tex_kwargs)
        title.center()
        title.scale(2)
        self.add(title)
        

In [ ]:
def make_cell(i=0, color="#E07A5F", size=0.8):
    if i == 0:
        return Square(color="#555", side_length=size)

    return VGroup(
        Square(color=color, side_length=size, fill_color=BLACK, fill_opacity=1),
        Text(str(i), color=color, size=size)
    )

def get_cell_value(cell):
    if len(cell) == 2:
        return int(cell[1].text)


def get_distance(n, cells):
    for i, cell in enumerate(cells):
        if get_cell_value(cell) == n:
            return abs(i % 4 - (n-1) % 4) + abs(i // 4 - (n-1) // 4)

@render
class MainScene(BaseScene):
    def make(self):
        random.seed(0)
        title = Tex(R"Heurística de Conflictos Lineales")
        title.center()

        cells = [make_cell(i) for i in range(16)]
        cell_ordered = cells.copy()  # un-suffle copy
        random.shuffle(cells)
    
        # Some changes to make a 2 lineal colicion
        cells[0], cells[11] = cells[11], cells[0] 
        cells[2], cells[5] = cells[5], cells[2]

        corrects_cells = [make_cell(i, color=GREEN) for i in range(1, 16)]
        base = [make_cell(0) for i in range(16)]

        game_grid = VGroup(*cells)
        game_grid.arrange_in_grid(rows=4, cols=4)
    
        corrects_cells_grid = VGroup(*corrects_cells)
        corrects_cells_grid.arrange_in_grid(rows=4, cols=4)

        base_grid = VGroup(*base)
        base_grid.arrange_in_grid(rows=4, cols=4)


        self.play(Write(title))
        self.wait(1)
        self.play(ApplyMethod(title.to_corner, UP + LEFT))

        self.play(ShowCreation(base_grid), ShowCreation(game_grid))
        self.wait(3)

        self.play(FadeIn(corrects_cells[0]), *[FadeOut(cell_ordered[i]) for i in range(16) if i != 1])
        
        distance_value = get_distance(1, cells)
        distance = Variable(distance_value, Tex("dist"), var_type=Integer)
        total_distance_value = distance_value
        total_distance = Variable(total_distance_value, Tex("total"), var_type=Integer)

        VGroup(distance, total_distance).arrange(DOWN).to_corner(LEFT)
        self.play(Write(distance), Write(total_distance))
    
        self.wait(3)
        self.remove(corrects_cells[0])
        self.remove(cell_ordered[1])
        
        for i in range(1, 15):
            self.add(corrects_cells[i])
            self.add(cell_ordered[i+1])
            cell_ordered[i+1].z_indez = 10
            corrects_cells[i].z_indez = 5
            distance_value = get_distance(i+1, cells)
            distance.tracker.set_value(distance_value)
            total_distance_value += distance_value
            self.play(ApplyMethod(total_distance.tracker.set_value, total_distance_value))
            self.wait(0.5)
            self.remove(corrects_cells[i])
            self.remove(cell_ordered[i+1])
        self.play(FadeOut(distance))
        self.wait(1)
 

        question = Tex(
            R"¿Qué pasa si consideramos en la relajación las piezas \\ cuyos",
            R" únicos caminos mínimos",
            R" se ",
            R" interceptan",
            R"?",
            **self.tex_kwargs
        )
        question[1].set_color(ORANGE)
        question[3].set_color(GREEN)
        question.to_corner(DOWN)
        self.play(Write(question))
    
        # Case 1
        indexes = [6, 14]
        self.play(*[FadeIn(corrects_cells[i-1]) for i in indexes] )
        self.play(*[FadeOut(corrects_cells[i-1]) for i in indexes] )
        self.play(*[FadeIn(cell_ordered[i]) for i in indexes] )
    
        shortest_paths = [
            DashedVMobject(Arrow(
                start=cell_ordered[i].get_center(),
                end=corrects_cells[i-1].get_center(),
                opacity=0.5,
                color=GREEN,
                max_tip_length_to_length_ratio=float("+inf"),
                max_stroke_width_to_length_ratio=float("+inf")
            )) for i in indexes
        ]
        for i, sp in enumerate(shortest_paths):
            sp.shift([0.2 * (2*i - 1) , 0, 0])
    
        self.add(*[p for p in shortest_paths])
        self.add(*[cell_ordered[i] for i in indexes])
        distance.tracker.set_value(sum(get_distance(i, cells) for i in indexes))
        self.play(*[Write(p) for p in shortest_paths], FadeIn(distance))
        self.wait(2)

        case_warning = Tex("¡Conflicto lineal!")
        case_warning.to_corner(RIGHT)
        case_heuristic = MathTex("+2").next_to(case_warning, DOWN)
        self.play(Write(case_warning))
        self.wait(3)
    
        c_points = list(map(VMobject.get_center, [base_grid[i] for i in [1, 2, 14, 13]]))
        path_points = []
        for cp1, cp2 in zip(c_points, c_points[1:]):
            if cp2 is c_points[-1]:
                path_points.append(DashedVMobject(Arrow(
                    cp1,
                    cp2 - 0.5 * (cp2 - cp1),
                    color=GREEN,
                    buff=0,
                    max_tip_length_to_length_ratio=float("+inf"),
                    max_stroke_width_to_length_ratio=float("+inf")
                )))
            else:
                path_points.append(DashedVMobject(Line(cp1, cp2, color=GREEN, stroke_width=5)))


        self.add(*path_points)
        self.add(*[cell_ordered[i] for i in indexes])
        self.play(*[FadeIn(p) for p in path_points], FadeOut(shortest_paths[1]))
        self.wait(2)

        self.play(Write(case_heuristic))
        total_distance_value += 2
        self.wait(1)

        self.play(ApplyMethod(total_distance.tracker.set_value, total_distance_value))
        self.wait(5)

        self.play(
            *[FadeOut(p) for p in [*path_points, shortest_paths[0], case_warning, case_heuristic, distance]],
            *[FadeOut(cell_ordered[i]) for i in indexes]
        )


        # Case 2
        indexes = [1, 2, 4]
        self.play(*[FadeIn(corrects_cells[i-1]) for i in indexes] )
        self.play(*[FadeOut(corrects_cells[i-1]) for i in indexes] )
        self.play(*[FadeIn(cell_ordered[i]) for i in indexes] )

        shortest_paths = [
            DashedVMobject(Arrow(
                start=cell_ordered[i].get_center(),
                end=corrects_cells[i-1].get_center(),
                opacity=0.5,
                color=GREEN,
                max_tip_length_to_length_ratio=float("+inf"),
                   max_stroke_width_to_length_ratio=float("+inf")
            )) for i in indexes
        ]


        self.add(*[p for p in shortest_paths])
        self.add(*[cell_ordered[i] for i in indexes])
        distance.tracker.set_value(sum(get_distance(i, cells) for i in indexes))
        self.play(*[Write(p) for p in shortest_paths], FadeIn(distance))
     
        case_warning = Tex("¡Conflictos lineales!")
        case_warning.to_corner(RIGHT)
        case_heuristic = MathTex(R"+ 2 \times 2").next_to(case_warning, DOWN)
        self.play(Write(case_warning))
        self.wait(3)
    

        c_points1 = list(map(VMobject.get_center, [base_grid[i] for i in [3, 7, 4, 0]]))
        c_points2 = list(map(VMobject.get_center, [base_grid[i] for i in [2, 6, 5, 1]]))
        path_points12 = []
        for c_points in [c_points1, c_points2]:
            c_group = []
            for cp1, cp2 in zip(c_points, c_points[1:]):
                if cp2 is c_points[-1]:
                    c_group.append(DashedVMobject(Arrow(
                        cp1,
                        cp2 - 0.5 * (cp2 - cp1),
                        color=GREEN,
                        buff=0,
                        max_tip_length_to_length_ratio=float("+inf"),
                        max_stroke_width_to_length_ratio=float("+inf")
                    )))
                else:
                    c_group.append(DashedVMobject(Line(cp1, cp2, color=GREEN, stroke_width=5)))
            path_points12.append(VGroup(*c_group))

        self.add(*path_points12)
        self.add(*[cell_ordered[i] for i in indexes])
        self.play(*[FadeIn(p) for p in path_points12], FadeOut(shortest_paths[0]), FadeOut(shortest_paths[1]))
        self.wait(2)

        self.play(Write(case_heuristic))
        total_distance_value += 4
        self.wait(1)

        self.play(ApplyMethod(total_distance.tracker.set_value, total_distance_value))
        self.wait(5)

        self.play(*[FadeOut(e) for e in
            [*path_points12, shortest_paths[2], case_warning, case_heuristic,
             case_warning, case_heuristic, question, distance, total_distance]
         ])

        # solve problem and exit
        for i in indexes:
            cell_ordered[i].save_state()

        actions = [(2, 7), (1, 8), (4, 2), (4, 3), (4, 4), (2, 6), (2, 2), (1, 7), (1, 6), (1, 5), (1, 1)]
        for act in actions:
            element, target = act
            shift_by = corrects_cells[target - 1].get_center() - cell_ordered[element].get_center()
            self.play(ApplyMethod(cell_ordered[element].shift, shift_by), play_time=4/11)

        self.play(*[FadeOut(cell_ordered[i]) for i in indexes], FadeOut(base_grid))

        for i in indexes:
            cell_ordered[i].restore()
        self.wait(5)
    
        # in light puzzle comment

        light_puzzle_notice = Tex(
            R"¡En el \textit{puzle liviano}, el costo a \\"
            R"resolver el conflicto depende de las piezas!"
        )
        light_puzzle_formula = MathTex(R"+ 2 \times \sum_{\text{confl}} (\text{cost mov})").to_corner(RIGHT)
        self.play(Write(light_puzzle_notice))
        self.wait(1)

        self.play(ApplyMethod(light_puzzle_notice.to_corner, DOWN))
        self.play(
            FadeIn(base_grid),
            *[FadeIn(p) for p in path_points12],
            FadeIn(shortest_paths[2]),
            *[FadeIn(cell_ordered[i]) for i in indexes],
        )
        self.wait(1)
        self.play(Wiggle(path_points12[0]))
        self.wait(1)
        self.play(Wiggle(path_points12[1]))
        self.wait(1)
        self.play(Write(light_puzzle_formula))
        self.wait(4)

        self.play(*[FadeOut(obj) for obj in self.mobjects])
